In [ ]:
# Imports
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
# Connect Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
SIZE = 224
IMAGE_SIZE = [224, 224]
INPUT_SHAPE = (SIZE, SIZE, 3) 
BATCH_SIZE = 32
DATASET_PATH = "/content/train_set/"
VALIDATION_PATH = "/content/val_set/"

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory(
        DATASET_PATH,
        target_size=(SIZE, SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        VALIDATION_PATH,
        target_size=(SIZE, SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 339995 images belonging to 7 classes.
Found 2987 images belonging to 7 classes.


In [ ]:
train_generator.class_indices

{'Anger': 0,
 'Disgust': 1,
 'Fear': 2,
 'Happy': 3,
 'Neutral': 4,
 'Sad': 5,
 'Surprise': 6}

# **Proposal Model**

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 activation (Activation)     (None, 222, 222, 64)      0         
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 dropout (Dropout)           (None, 109, 109, 128)     0

In [ ]:
CHECKPOINT_PATH = "/content/saved_models/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model_without_pretrain = model.fit(
        train_generator,
        epochs=15,
        batch_size=BATCH_SIZE,
        validation_data=validation_generator,
        callbacks=callbacks_list)

model.save(CHECKPOINT_PATH+"emotion_model.h5")

Epoch 1/15
10625/10625 [==============================] - ETA: 0s - loss: 0.9094 - accuracy: 0.6687
Epoch 00001: saving model to /content/saved_models2/weights-improvement-01-0.406.hdf5
10625/10625 [==============================] - 2926s 274ms/step - loss: 0.9094 - accuracy: 0.6687 - val_loss: 1.8950 - val_accuracy: 0.4061
Epoch 2/15
10625/10625 [==============================] - ETA: 0s - loss: 0.6608 - accuracy: 0.7548
Epoch 00002: saving model to /content/saved_models2/weights-improvement-02-0.420.hdf5
10625/10625 [==============================] - 2898s 273ms/step - loss: 0.6608 - accuracy: 0.7548 - val_loss: 2.0237 - val_accuracy: 0.4205
Epoch 3/15
10625/10625 [==============================] - ETA: 0s - loss: 0.5770 - accuracy: 0.7853
Epoch 00003: saving model to /content/saved_models2/weights-improvement-03-0.438.hdf5
10625/10625 [==============================] - 2891s 272ms/step - loss: 0.5770 - accuracy: 0.7853 - val_loss: 2.3727 - val_accuracy: 0.4376
Epoch 4/15
10625/10625

In [ ]:
loaded_model = load_model('/content/weights-improvement-08-0.446.hdf5')
model_with_pretrain = loaded_model.fit(
        train_generator,
        epochs=15,
        initial_epoch=8,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        callbacks=callbacks_list)
model.save(CHECKPOINT_PATH+"emotion_model.h5")


Epoch 9/15
10625/10625 [==============================] - ETA: 0s - loss: 0.2629 - accuracy: 0.9002
Epoch 00009: saving model to /content/saved_models/weights-improvement-09-0.432.hdf5
10625/10625 [==============================] - 2985s 280ms/step - loss: 0.2629 - accuracy: 0.9002 - val_loss: 4.1743 - val_accuracy: 0.4319
Epoch 10/15
10625/10625 [==============================] - ETA: 0s - loss: 0.2325 - accuracy: 0.9120
Epoch 00010: saving model to /content/saved_models/weights-improvement-10-0.401.hdf5
10625/10625 [==============================] - 2975s 280ms/step - loss: 0.2325 - accuracy: 0.9120 - val_loss: 3.7745 - val_accuracy: 0.4011
Epoch 11/15
10625/10625 [==============================] - ETA: 0s - loss: 0.2065 - accuracy: 0.9229
Epoch 00011: saving model to /content/saved_models/weights-improvement-11-0.435.hdf5
10625/10625 [==============================] - 2975s 280ms/step - loss: 0.2065 - accuracy: 0.9229 - val_loss: 2.9475 - val_accuracy: 0.4349
Epoch 12/15
10625/10625

In [ ]:
loaded_model = load_model('/content/weights-improvement-13-0.450.hdf5')
model_with_pretrain = loaded_model.fit(
        train_generator,
        epochs=15,
        initial_epoch=13,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        callbacks=callbacks_list)
model.save(CHECKPOINT_PATH+"emotion_model.h5")

Epoch 14/15
10625/10625 [==============================] - ETA: 0s - loss: 0.1565 - accuracy: 0.9422
Epoch 00014: saving model to /content/saved_models2/weights-improvement-14-0.431.hdf5
10625/10625 [==============================] - 3703s 347ms/step - loss: 0.1565 - accuracy: 0.9422 - val_loss: 2.8692 - val_accuracy: 0.4305
Epoch 15/15
10625/10625 [==============================] - ETA: 0s - loss: 0.1428 - accuracy: 0.9474
Epoch 00015: saving model to /content/saved_models2/weights-improvement-15-0.417.hdf5
10625/10625 [==============================] - 3678s 346ms/step - loss: 0.1428 - accuracy: 0.9474 - val_loss: 5.2753 - val_accuracy: 0.4168


NameError: ignored

# **MobileNet**

In [ ]:
import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization   
mobileNet=keras.applications.mobilenet_v2.MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in mobileNet.layers[:1]:
    layer.trainable = False

mb=Flatten()(mobileNet.output)
mb=Dense(128,activation='relu')(mb)
mb=BatchNormalization()(mb)

mb=Dense(64,activation='relu')(mb)
mb=BatchNormalization()(mb)
prediction = Dense(7, activation='softmax')(mb)

# create a model object
model_mb = Model(inputs=mobileNet.input, outputs=prediction)

# view the structure of the model
model_mb.summary()

# tell the model what cost and optimization method to use
model_mb.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
CHECKPOINT_PATH = "/content/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"mobilenet/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
callbacks_list = [checkpoint,early_stop]

# fit the model
model_with_mb = model_mb.fit(
        train_generator,
        epochs=15,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_mb.save(CHECKPOINT_PATH+"mb_balanced.h5")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 3734s 350ms/step - loss: 0.6156 - accuracy: 0.7660 - val_loss: 2.7951 - val_accuracy: 0.3569
Epoch 2/15
10625/10625 [==============================] - ETA: 0s - loss: 0.5305 - accuracy: 0.7987
Epoch 00002: val_accuracy improved from 0.35688 to 0.43589, saving model to /content/mobilenet/weights-improvement-02-0.436.hdf5
10625/10625 [==============================] - 3679s 346ms/step - loss: 0.5305 - accuracy: 0.7987 - val_loss: 2.8886 - val_accuracy: 0.4359
Epoch 3/15
10625/10625 [==============================] - ETA: 0s - loss: 0.4938 - accuracy: 0.8126
Epoch 00003: val_accuracy improved from 0.43589 to 0.47941, saving model to /content/mobilenet/weights-improvement-03-0.479.hdf5
10625/10625 [==============================] - 3690s 347ms/step - loss: 0.4938 - accuracy: 0.8126 - val_loss: 2.9616 - val_accuracy: 0.4794
Epoch 00003: early stopping


In [ ]:
CHECKPOINT_PATH = "/content/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"mobilenet/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


model_with_mb = load_model('/content/mobilenet/weights-improvement-03-0.479.hdf5')
model_with_mobileNet = model_with_mb.fit(
        train_generator,
        epochs=15,
        initial_epoch=3,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

Epoch 4/15
10625/10625 [==============================] - ETA: 0s - loss: 0.4679 - accuracy: 0.8219
Epoch 00004: saving model to /content/mobilenet/weights-improvement-04-0.467.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 3703s 348ms/step - loss: 0.4679 - accuracy: 0.8219 - val_loss: 2.4102 - val_accuracy: 0.4674
Epoch 5/15
 9936/10625 [===========================>..] - ETA: 3:59 - loss: 0.4516 - accuracy: 0.8278

In [ ]:
model_with_mb = load_model('/content/mobilenet-weights-improvement-04-0.467.hdf5')
model_with_mobileNet = model_with_mb.fit(
        train_generator,
        epochs=15,
        initial_epoch=4,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
model_with_mb.save(CHECKPOINT_PATH+"mb_balanced.h5")

Epoch 5/15
10625/10625 [==============================] - ETA: 0s - loss: 0.4520 - accuracy: 0.8269
Epoch 00005: saving model to /content/mobilenet/mobilenet-weights-improvement-05-0.479.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 3880s 363ms/step - loss: 0.4520 - accuracy: 0.8269 - val_loss: 3.2671 - val_accuracy: 0.4787
Epoch 6/15
10625/10625 [==============================] - ETA: 0s - loss: 0.4356 - accuracy: 0.8333
Epoch 00006: saving model to /content/mobilenet/mobilenet-weights-improvement-06-0.453.hdf5
10625/10625 [==============================] - 3871s 364ms/step - loss: 0.4356 - accuracy: 0.8333 - val_loss: 2.8316 - val_accuracy: 0.4530
Epoch 7/15
10625/10625 [==============================] - ETA: 0s - loss: 0.4214 - accuracy: 0.8389
Epoch 00007: saving model to /content/mobilenet/mobilenet-weights-improvement-07-0.468.hdf5
10625/10625 [==============================] - 3832s 361ms/step - loss: 0.4214 - accuracy: 0.8389 - val_loss: 3.3801 - val_accuracy: 0.4677
Epoch 8/15
 7791/10625 [====================>.........] - ETA: 16:51 - loss: 0.4072 - accuracy: 0.8434

In [ ]:
model_with_mb = load_model('/content/mobilenet-weights-improvement-07-0.468.hdf5')
model_with_mobileNet = model_with_mb.fit(
        train_generator,
        epochs=15,
        initial_epoch=7,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
model_with_mb.save(CHECKPOINT_PATH+"mb_balanced.h5")

Epoch 8/15
10625/10625 [==============================] - ETA: 0s - loss: 0.4076 - accuracy: 0.8434
Epoch 00008: saving model to /content/mobilenet/mobilenet-weights-improvement-08-0.492.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 5088s 477ms/step - loss: 0.4076 - accuracy: 0.8434 - val_loss: 2.4217 - val_accuracy: 0.4918
Epoch 9/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3948 - accuracy: 0.8481
Epoch 00009: saving model to /content/mobilenet/mobilenet-weights-improvement-09-0.459.hdf5
10625/10625 [==============================] - 5090s 479ms/step - loss: 0.3948 - accuracy: 0.8481 - val_loss: 3.2585 - val_accuracy: 0.4587
Epoch 10/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3827 - accuracy: 0.8525
Epoch 00010: saving model to /content/mobilenet/mobilenet-weights-improvement-10-0.456.hdf5
10625/10625 [==============================] - 5062s 476ms/step - loss: 0.3827 - accuracy: 0.8525 - val_loss: 3.7220 - val_accuracy: 0.4563
Epoch 11/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3697 - accuracy: 0.8573
Epoch 00011: saving model to /content/mobilenet/mobilenet-weights-improvement-11-0.466.hdf5
10

In [ ]:
model_with_mb = load_model('/content/mobilenet-weights-improvement-11-0.466.hdf5')
model_with_mobileNet = model_with_mb.fit(
        train_generator,
        epochs=15,
        initial_epoch=11,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
model_with_mb.save(CHECKPOINT_PATH+"mb_balanced.h5")

Epoch 12/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3583 - accuracy: 0.8610
Epoch 00012: saving model to /content/mobilenet/mobilenet-weights-improvement-12-0.470.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 3740s 351ms/step - loss: 0.3583 - accuracy: 0.8610 - val_loss: 3.5673 - val_accuracy: 0.4700
Epoch 13/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3466 - accuracy: 0.8654
Epoch 00013: saving model to /content/mobilenet/mobilenet-weights-improvement-13-0.464.hdf5
10625/10625 [==============================] - 3682s 347ms/step - loss: 0.3466 - accuracy: 0.8654 - val_loss: 4.0363 - val_accuracy: 0.4643
Epoch 14/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3355 - accuracy: 0.8693
Epoch 00014: saving model to /content/mobilenet/mobilenet-weights-improvement-14-0.484.hdf5
10625/10625 [==============================] - 3667s 345ms/step - loss: 0.3355 - accuracy: 0.8693 - val_loss: 3.1714 - val_accuracy: 0.4841
Epoch 15/15
10625/10625 [==============================] - ETA: 0s - loss: 0.3243 - accuracy: 0.8736
Epoch 00015: saving model to /content/mobilenet/mobilenet-weights-improvement-15-0.465.hdf5
1

# **ResNet**

In [ ]:
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization    
resnet = resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
for layer in resnet.layers[:3]:
    layer.trainable = False

# our layers 
r=Flatten()(resnet.output)
r=Dense(128,activation='relu')(r)
r=BatchNormalization()(r)
r=Dropout(0.25)(r)

r=Dense(64,activation='relu')(r)
r=BatchNormalization()(r)
prediction = Dense(7, activation='softmax')(r)
# create a model object
model_res = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model_res.summary()

# tell the model what cost and optimization method to use
model_res.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

CHECKPOINT_PATH = "/content/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"resNet/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# fit the model
model_with_resNet = model_res.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_res.save(CHECKPOINT_PATH+"res_balanced.h5")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 7596s 714ms/step - loss: 0.7074 - accuracy: 0.7287 - val_loss: 3.2346 - val_accuracy: 0.4386
Epoch 2/10
10625/10625 [==============================] - ETA: 0s - loss: 0.5680 - accuracy: 0.7834
Epoch 00002: saving model to /content/resNet/weights-improvement-02-0.428.hdf5
10625/10625 [==============================] - 7544s 710ms/step - loss: 0.5680 - accuracy: 0.7834 - val_loss: 3.3205 - val_accuracy: 0.4275
Epoch 3/10
10220/10625 [===========================>..] - ETA: 4:46 - loss: 0.5161 - accuracy: 0.8036

In [ ]:
checkpoint = ModelCheckpoint("/content/resnet-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


resnet = load_model('/content/resnet-weights-improvement-02-0.428.hdf5')
model_with_resnet = resnet.fit(
        train_generator,
        epochs=10,
        initial_epoch=2,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
resnet.save(CHECKPOINT_PATH+"resnet_imbalanced.h5")

Epoch 3/10
10625/10625 [==============================] - ETA: 0s - loss: 0.5171 - accuracy: 0.8032
Epoch 00003: saving model to /content/resnet-weights-improvement-03-0.433.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 7414s 696ms/step - loss: 0.5171 - accuracy: 0.8032 - val_loss: 3.3899 - val_accuracy: 0.4325
Epoch 4/10
 7989/10625 [=====================>........] - ETA: 30:28 - loss: 0.4784 - accuracy: 0.8168

In [ ]:
resnet = load_model('/content/resnet-weights-improvement-03-0.433.hdf5')
model_with_resnet = resnet.fit(
        train_generator,
        epochs=10,
        initial_epoch=3,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
resnet.save(CHECKPOINT_PATH+"resnet_imbalanced.h5")

Epoch 4/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4800 - accuracy: 0.8171
Epoch 00004: saving model to /content/resnet-weights-improvement-04-0.442.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 7480s 703ms/step - loss: 0.4800 - accuracy: 0.8171 - val_loss: 3.5500 - val_accuracy: 0.4419
Epoch 5/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4458 - accuracy: 0.8292
Epoch 00005: saving model to /content/resnet-weights-improvement-05-0.436.hdf5
10625/10625 [==============================] - 7497s 706ms/step - loss: 0.4458 - accuracy: 0.8292 - val_loss: 3.3826 - val_accuracy: 0.4362
Epoch 6/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4074 - accuracy: 0.8439

# **Effnet**

In [ ]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization    
effNet= EfficientNetB1(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)

for layer in effNet.layers[:5]:
    layer.trainable = False

# our layers - you can add more if you want
ef=Flatten()(effNet.output)
ef=Dense(128,activation='relu')(ef)
ef=BatchNormalization()(ef)
ef=Dropout(0.25)(ef)

ef=Dense(64,activation='relu')(ef)
ef=BatchNormalization()(ef)
prediction = Dense(7, activation='softmax')(ef)

# create a model object
model_eff = Model(inputs=effNet.input, outputs=prediction)

# view the structure of the model
model_eff.summary()

# tell the model what cost and optimization method to use
model_eff.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"effNet/weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
# early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
callbacks_list = [checkpoint]


# fit the model
model_with_effNet = model_eff.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_eff.save(CHECKPOINT_PATH+"eff_imbalanced.h5")

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_3 (Rescaling)        (None, 224, 224, 3)  0           ['input_4[0][0]']                
                                                                                                  
 normalization_3 (Normalization  (None, 224, 224, 3)  7          ['rescaling_3[0][0]']            
 )                                                                                                
                                                                                            

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 8752s 821ms/step - loss: 0.6082 - accuracy: 0.7713 - val_loss: 3.3460 - val_accuracy: 0.3934
Epoch 2/50
10625/10625 [==============================] - ETA: 0s - loss: 0.5134 - accuracy: 0.8059
Epoch 00002: saving model to /content/saved_models/effNet/weights-improvement-02-0.352.hdf5
10625/10625 [==============================] - 8617s 811ms/step - loss: 0.5134 - accuracy: 0.8059 - val_loss: 2.9258 - val_accuracy: 0.3515
Epoch 3/50
 1117/10625 [==>...........................] - ETA: 2:08:15 - loss: 0.4656 - accuracy: 0.8252

In [ ]:
checkpoint = ModelCheckpoint("/content/effnet-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# fit the model
effnet = load_model('/content/effnet-weights-improvement-02-0.352.hdf5')
model_with_effNet = effnet.fit(
        train_generator,
        epochs=30,
        initial_epoch=2,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

Epoch 3/30
10625/10625 [==============================] - ETA: 0s - loss: 0.4788 - accuracy: 0.8190
Epoch 00003: saving model to /content/effnet-weights-improvement-03-0.457.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 8808s 827ms/step - loss: 0.4788 - accuracy: 0.8190 - val_loss: 3.1859 - val_accuracy: 0.4573
Epoch 4/30
10625/10625 [==============================] - ETA: 0s - loss: 0.4528 - accuracy: 0.8285
Epoch 00004: saving model to /content/effnet-weights-improvement-04-0.459.hdf5
10625/10625 [==============================] - 8769s 825ms/step - loss: 0.4528 - accuracy: 0.8285 - val_loss: 4.5599 - val_accuracy: 0.4593
Epoch 5/30
 1589/10625 [===>..........................] - ETA: 2:03:58 - loss: 0.4193 - accuracy: 0.8402

In [ ]:
CHECKPOINT_PATH = "/content/saved_models/"
checkpoint = ModelCheckpoint("/content/effnet-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint, early_stop]


effnet = load_model('/content/effnet-weights-improvement-04-0.459.hdf5')
model_with_effNet = effnet.fit(
        train_generator,
        epochs=30,
        initial_epoch=4,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

Epoch 5/30
10625/10625 [==============================] - ETA: 0s - loss: 0.4310 - accuracy: 0.8369
Epoch 00005: saving model to /content/effnet-weights-improvement-05-0.463.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


10625/10625 [==============================] - 8767s 823ms/step - loss: 0.4310 - accuracy: 0.8369 - val_loss: 3.3399 - val_accuracy: 0.4630
Epoch 6/30
10625/10625 [==============================] - ETA: 0s - loss: 0.4114 - accuracy: 0.8435
Epoch 00006: saving model to /content/effnet-weights-improvement-06-0.452.hdf5
10625/10625 [==============================] - 8725s 821ms/step - loss: 0.4114 - accuracy: 0.8435 - val_loss: 3.4608 - val_accuracy: 0.4516
Epoch 7/30
 6721/10625 [=================>............] - ETA: 53:42 - loss: 0.3918 - accuracy: 0.8500

# **VGG**

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import tensorflow
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization

#preprocessing part

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights

for layer in vgg.layers[:5]:
    layer.trainable = False

# our layers - you can add more if you want
v=Flatten()(vgg.output)
v=Dense(128,activation='relu')(v)
v=BatchNormalization()(v)
v=Dropout(0.25)(v)

v=Dense(64,activation='relu')(v)
v=BatchNormalization()(v)
prediction = Dense(7, activation='softmax')(v)

# create a model object
model_vgg = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model_vgg.summary()

# tell the model what cost and optimization method to use
model_vgg.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

CHECKPOINT_PATH="/content/"
checkpoint = ModelCheckpoint(CHECKPOINT_PATH+"vgg/vgg-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


# fit the model
model_with_vgg = model_vgg.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)

model_vgg.save(CHECKPOINT_PATH+"vgg_balanced.h5")

58900480/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
checkpoint = ModelCheckpoint("/content/vgg-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


vgg = load_model('/content/vgg-weights-improvement-02-0.415.hdf5')
model_with_vgg = vgg.fit(
        train_generator,
        epochs=10,
        initial_epoch=2,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
vgg.save(CHECKPOINT_PATH+"vgg_imbalanced.h5")

Epoch 3/10
11101/11101 [==============================] - ETA: 0s - loss: 0.6679 - accuracy: 0.7408
Epoch 00003: saving model to /content/vgg-weights-improvement-03-0.389.hdf5
11101/11101 [==============================] - 6286s 566ms/step - loss: 0.6679 - accuracy: 0.7408 - val_loss: 4.6759 - val_accuracy: 0.3890
Epoch 4/10
11101/11101 [==============================] - ETA: 0s - loss: 0.6302 - accuracy: 0.7560
Epoch 00004: saving model to /content/vgg-weights-improvement-04-0.416.hdf5
11101/11101 [==============================] - 6273s 565ms/step - loss: 0.6302 - accuracy: 0.7560 - val_loss: 2.7605 - val_accuracy: 0.4161
Epoch 5/10
 1185/11101 [==>...........................] - ETA: 1:33:25 - loss: 0.6057 - accuracy: 0.7642

KeyboardInterrupt: ignored

In [ ]:
checkpoint = ModelCheckpoint("/content/vgg-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


vgg = load_model('/content/vgg-weights-improvement-04-0.416.hdf5')
model_with_vgg = vgg.fit(
        train_generator,
        epochs=10,
        initial_epoch=4,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
vgg.save(CHECKPOINT_PATH+"vgg_imbalanced.h5")

Epoch 5/10
10625/10625 [==============================] - ETA: 0s - loss: 0.5269 - accuracy: 0.8004
Epoch 00005: saving model to /content/vgg-weights-improvement-05-0.460.hdf5
10625/10625 [==============================] - 5984s 561ms/step - loss: 0.5269 - accuracy: 0.8004 - val_loss: 3.5559 - val_accuracy: 0.4600
Epoch 6/10
10625/10625 [==============================] - ETA: 0s - loss: 0.5034 - accuracy: 0.8092
Epoch 00006: saving model to /content/vgg-weights-improvement-06-0.476.hdf5
10625/10625 [==============================] - 5940s 559ms/step - loss: 0.5034 - accuracy: 0.8092 - val_loss: 2.3339 - val_accuracy: 0.4757
Epoch 7/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4843 - accuracy: 0.8159
Epoch 00007: saving model to /content/vgg-weights-improvement-07-0.454.hdf5
10625/10625 [==============================] - 5919s 557ms/step - loss: 0.4843 - accuracy: 0.8159 - val_loss: 3.2060 - val_accuracy: 0.4536
Epoch 8/10
 6773/10625 [==================>.........

In [ ]:
checkpoint = ModelCheckpoint("/content/vgg-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


vgg = load_model('/content/vgg-weights-improvement-07-0.454.hdf5')
model_with_vgg = vgg.fit(
        train_generator,
        epochs=10,
        initial_epoch=7,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
vgg.save(CHECKPOINT_PATH+"vgg_imbalanced.h5")

Epoch 8/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4666 - accuracy: 0.8221
Epoch 00008: saving model to /content/vgg-weights-improvement-08-0.457.hdf5
10625/10625 [==============================] - 5707s 535ms/step - loss: 0.4666 - accuracy: 0.8221 - val_loss: 3.7550 - val_accuracy: 0.4570
Epoch 9/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4498 - accuracy: 0.8289
Epoch 00009: saving model to /content/vgg-weights-improvement-09-0.444.hdf5
10625/10625 [==============================] - 5667s 533ms/step - loss: 0.4498 - accuracy: 0.8289 - val_loss: 3.5410 - val_accuracy: 0.4439
Epoch 10/10
10404/10625 [============================>.] - ETA: 1:57 - loss: 0.4321 - accuracy: 0.8359

In [ ]:
checkpoint = ModelCheckpoint("/content/vgg-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]


vgg = load_model('/content/vgg-weights-improvement-09-0.444.hdf5')
model_with_vgg = vgg.fit(
        train_generator,
        epochs=10,
        initial_epoch=9,
        validation_data=validation_generator,
        batch_size = BATCH_SIZE,
        steps_per_epoch = len(train_generator),
        validation_steps = len(validation_generator),
        callbacks=callbacks_list)
vgg.save(CHECKPOINT_PATH+"vgg_imbalanced.h5")

Epoch 10/10
10625/10625 [==============================] - ETA: 0s - loss: 0.4329 - accuracy: 0.8353
Epoch 00010: saving model to /content/vgg-weights-improvement-10-0.452.hdf5
10625/10625 [==============================] - 5767s 541ms/step - loss: 0.4329 - accuracy: 0.8353 - val_loss: 4.3404 - val_accuracy: 0.4516


NameError: ignored

In [ ]:
vgg.save("content/vgg_imbalanced.h5")